In [1]:
import json
import pandas as pd

# data = []
# with open("../data/debate.jsonl", "r") as f:
#     for line in f:
#         data.append(json.loads(line))

In [2]:
data = []

with open("../data/claim_topics_2.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
data

[{'claim': '. I believe in same sex and traditional marriage but not other types of marriage',
  'topic': 'Same-sex marriage'},
 {'claim': 'A direct online democracy where people vote on issues instead of candidates would be better than always having to vote for the less bad of candidates.',
  'topic': 'Electronic voting'},
 {'claim': 'A financial abortion law for men is feasible',
  'topic': 'Abortion'},
 {'claim': 'A teenager can be just as intelligent as an adult on certain topics because age and experience dont equal intelligence.',
  'topic': 'Sex differences in intelligence'},
 {'claim': 'A vegan diet that requires supplements is neither natural nor healthy. .',
  'topic': 'Veganism'},
 {'claim': 'A year prison sentence is just as bad as a life sentence',
  'topic': 'Prison'},
 {'claim': 'Abortion I dont believe you can harm someone by failing to cause them to exist .',
  'topic': 'Abortion'},
 {'claim': 'Abortion is justifiable but most abortions are immoral',
  'topic': 'Aborti

In [4]:
import requests

with open("../creds.json", "r") as f:
    config = json.load(f)

In [5]:
# Try Aspects
# Note: COPIED CODE

def identify_aspects(topic, claim):
    # Note: COPIED CODE
    payload = {
        "query": topic,
        "arguments": claim,
        "userID": config["argumentext_user"],
        "apiKey": config["argumentext_api_key"],
    }

    is_timed_out = True
    timed_out_ctr = 1
    while is_timed_out == True:
        try:
            json_dict = requests.post("https://api.argumentsearch.com/en/get_aspects", timeout=300, data=json.dumps(payload),
                                      headers={'Content-Type': 'application/json'}).json()
            is_timed_out = False
        except requests.exceptions.Timeout or ConnectionError or json.decoder.JSONDecodeError:
            print("Timed out for {0} times.".format(str(timed_out_ctr)))
        except Exception as e:
            print(e)

    return json_dict

In [6]:
def identify_stance(topic, claim):
    # Note: COPIED CODE
    payload = {
        "topic": topic,
        "text": claim,
        "predictStance": True,
        "userID": config["argumentext_user"],
        "apiKey": config["argumentext_api_key"],
    }

    is_timed_out = True
    timed_out_ctr = 1
    while is_timed_out == True:
        try:
            json_dict = requests.post("https://api.argumentsearch.com/en/classify", timeout=300, data=json.dumps(payload),
                                      headers={'Content-Type': 'application/json'}).json()
            is_timed_out = False
        except requests.exceptions.Timeout or ConnectionError or json.decoder.JSONDecodeError:
            print("Timed out for {0} times.".format(str(timed_out_ctr)))
        except Exception as e:
            print(e)

    return json_dict


In [10]:
arg_units = {}

for i in data:
    arg_units.update({
    i["claim"]:
        {
            "topic": i["topic"],
            "stance": "",
            "aspect": "",
            }})

arg_units

{'. I believe in same sex and traditional marriage but not other types of marriage': {'topic': 'Same-sex marriage',
  'stance': '',
  'aspect': ''},
 'A direct online democracy where people vote on issues instead of candidates would be better than always having to vote for the less bad of candidates.': {'topic': 'Electronic voting',
  'stance': '',
  'aspect': ''},
 'A financial abortion law for men is feasible': {'topic': 'Abortion',
  'stance': '',
  'aspect': ''},
 'A teenager can be just as intelligent as an adult on certain topics because age and experience dont equal intelligence.': {'topic': 'Sex differences in intelligence',
  'stance': '',
  'aspect': ''},
 'A vegan diet that requires supplements is neither natural nor healthy. .': {'topic': 'Veganism',
  'stance': '',
  'aspect': ''},
 'A year prison sentence is just as bad as a life sentence': {'topic': 'Prison',
  'stance': '',
  'aspect': ''},
 'Abortion I dont believe you can harm someone by failing to cause them to exist

In [11]:
for i in data:
    stance = identify_stance(i["topic"], i["claim"])
    aspect = identify_aspects(i["topic"], {"1": i["claim"]})

    if aspect["aspects"]:
        arg_units[i["claim"]]["aspect"] = str(aspect["aspects"][0])

    if stance["sentences"]:
        arg_units[i["claim"]]["stance"] = stance["sentences"][0]["stanceLabel"]

arg_units

{'. I believe in same sex and traditional marriage but not other types of marriage': {'topic': 'Same-sex marriage',
  'stance': '',
  'aspect': 'traditional marriage'},
 'A direct online democracy where people vote on issues instead of candidates would be better than always having to vote for the less bad of candidates.': {'topic': 'Electronic voting',
  'stance': 'pro',
  'aspect': 'online democracy'},
 'A financial abortion law for men is feasible': {'topic': 'Abortion',
  'stance': 'pro',
  'aspect': 'financial abortion law'},
 'A teenager can be just as intelligent as an adult on certain topics because age and experience dont equal intelligence.': {'topic': 'Sex differences in intelligence',
  'stance': 'pro',
  'aspect': 'age'},
 'A vegan diet that requires supplements is neither natural nor healthy. .': {'topic': 'Veganism',
  'stance': 'contra',
  'aspect': 'natural'},
 'A year prison sentence is just as bad as a life sentence': {'topic': 'Prison',
  'stance': 'pro',
  'aspect':

In [12]:
arg_units_ = []

for i in arg_units:
    arg_units_.append({
        "claim": i,
        "topic": arg_units[i]["topic"],
        "stance": arg_units[i]["stance"],
        "aspect": arg_units[i]["aspect"]
    })

with open("../data/argument_units.jsonl", "w") as f:
    for line in arg_units_:
        f.write(json.dumps(line))
        f.write("\n")

In [15]:
arg_units_

[{'claim': '. I believe in same sex and traditional marriage but not other types of marriage',
  'topic': 'Same-sex marriage',
  'stance': '',
  'aspect': 'traditional marriage'},
 {'claim': 'A direct online democracy where people vote on issues instead of candidates would be better than always having to vote for the less bad of candidates.',
  'topic': 'Electronic voting',
  'stance': 'pro',
  'aspect': 'online democracy'},
 {'claim': 'A financial abortion law for men is feasible',
  'topic': 'Abortion',
  'stance': 'pro',
  'aspect': 'financial abortion law'},
 {'claim': 'A teenager can be just as intelligent as an adult on certain topics because age and experience dont equal intelligence.',
  'topic': 'Sex differences in intelligence',
  'stance': 'pro',
  'aspect': 'age'},
 {'claim': 'A vegan diet that requires supplements is neither natural nor healthy. .',
  'topic': 'Veganism',
  'stance': 'contra',
  'aspect': 'natural'},
 {'claim': 'A year prison sentence is just as bad as a l

In [437]:
### KB QUERY ###

from neo4j import GraphDatabase

class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None

        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))

        except Exception as e:
            print("Failed to create driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None

        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))

        except Exception as e:
            print("Query failed:", e)

        finally:
            if session is not None:
                session.close()

            return response

In [439]:
uri = "bolt://127.0.0.1:7687"
user = "neo4j"
pwd = "testing"

# conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)

In [441]:
graphdp = GraphDatabase.driver(uri=uri, auth=(user,pwd))
session = graphdp.session()

claim = "Women deserve the right to an abortion. It's thier choice"
topic = ["abortion"]

results = []

for i in topic:
    query_string_2 ="MATCH (n:CausalConcept {concept:$i})-[c]-(b)-[e]-(f:CausalConcept)" \
                                                               "MATCH (b)-[]-(h)" \
                                                               "RETURN n, b, e, c, f, h"
    nodes = session.run(query_string_2, i=i)
    results.append((i, nodes))

query, nodes_ = results[0]
results_ = [i for i in nodes_.data()]

In [445]:
results_

[{'n': {'concept': 'abortion'},
  'b': {'type': 'mayCause'},
  'e': ({'type': 'mayCause'}, 'effect', {'concept': 'death'}),
  'c': ({'type': 'mayCause'}, 'cause', {'concept': 'abortion'}),
  'f': {'concept': 'death'},
  'h': {'concept': 'abortion'}},
 {'n': {'concept': 'abortion'},
  'b': {'type': 'mayCause'},
  'e': ({'type': 'mayCause'}, 'effect', {'concept': 'death'}),
  'c': ({'type': 'mayCause'}, 'cause', {'concept': 'abortion'}),
  'f': {'concept': 'death'},
  'h': {'concept': 'death'}},
 {'n': {'concept': 'abortion'},
  'b': {'type': 'mayCause'},
  'e': ({'type': 'mayCause'}, 'effect', {'concept': 'death'}),
  'c': ({'type': 'mayCause'}, 'cause', {'concept': 'abortion'}),
  'f': {'concept': 'death'},
  'h': {'sentence': 'Abortion can lead to death if serious complications arise.',
   'clueweb12_page_timestamp': '2012-04-05T15:49:48Z',
   'path_pattern': '[[cause]]/N\t-nsubj\tlead/VB\t+nmod:to\t[[effect]]/N',
   'clueweb12_page_reference': 'http://www.articlehotline.com/articles/

In [472]:
# Cache Cause-Effect Triples
concepts = set()

for i in range(0, len(results_)):
    type, relation, concept = results_[i]["e"]
    concept = concept["concept"].replace("_", "")

    concepts.add(f"{concept} {relation} {query}")

evidence =  dict((k, []) for k in concepts)

for i in range(0, len(results_)):
    type, relation, concept = results_[i]["e"]
    concept = concept["concept"].replace("_", "")

    concept_ = f"{concept} {relation} {query}"

    if results_[i]["h"]:
        ev = next(iter(results_[i]["h"].values()))
        evidence[concept_].append(ev)

    else:
        continue

triples = [str(i) for i in evidence.keys()]
triples

['infections effect abortion',
 'mifepristone cause abortion',
 'harmfulpsychologicaleffects effect abortion',
 'distress effect abortion',
 'harmtowomen effect abortion',
 'problems effect abortion',
 'emotionaltrauma effect abortion',
 'unintendedpregnancy cause abortion',
 'brucellosis cause abortion',
 'suicide effect abortion',
 'infection cause abortion',
 'emergencycontraception cause abortion',
 'pain effect abortion',
 'maternalmortality effect abortion',
 'deaths effect abortion',
 'substanceabuse effect abortion',
 'death effect abortion',
 'suffering effect abortion',
 'pregnancy cause abortion',
 'pretermbirth effect abortion',
 'rape cause abortion',
 'stress cause abortion',
 'depression effect abortion',
 'infertility effect abortion',
 'planb cause abortion',
 'increase effect abortion',
 'drug cause abortion',
 'cancer effect abortion',
 'birthcontrol cause abortion',
 'mentalillness effect abortion',
 'misoprostol cause abortion',
 'infection effect abortion',
 'emot

In [466]:
from transformers import pipeline
import numpy as np
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def semantic_entailment(claim, statements, classifier=classifier):
    entails = classifier(claim, statements)

    idx_min = np.argmin(entails["scores"])
    idx_max = np.argmax(entails["scores"])

    entails_ = statements[idx_min]
    counter = statements[idx_max]

    return counter

In [ ]:
entailment = semantic_entailment(claim, triples)

In [502]:
counter_evidence_ = evidence[entailment]
len(counter_evidence_)

7

In [504]:
### NLG ###
reply = []

counter_evidence_ = counter_evidence_[0:5]

for i in range(0, len(counter_evidence_)):

    if (i < len(counter_evidence_) - 1):
        reply.append(f"{counter_evidence_[i]} and ")

    else:
        reply.append(f"{counter_evidence_[i]}")

literal = ""
for i in reply:
    literal += "".join(i)

literal

'Infections caused by an abortion or miscarriage can also cause scar tissue to develop. and Abortion can lead to infections (such as pelvic inflammatory disease), as well as uterine scarring. and Abortion can result in infections and damage to the cervix and uterus. and 16 This is because abortion can lead to infections (such as pelvic inflammatory disease), as well as uterine scarring. and 16 This is because abortion can lead to infections (such as pelvic inflammatory disease), as well as uterine scarring.'

In [510]:
counter_argument = ""

entailment_ = entailment.split(" ")

if entailment_[1] == "effect":
    counter_argument = f"{entailment_[0]} can lead to {entailment_[2]}"

else:
    counter_argument = f"{entailment_[2]} can lead to {entailment_[0]}"

counter_argument

'infections can lead to abortion'

In [ ]:
# TODOs NLI Textual Entailment
# Consider suitable NLI Model and dataset
# TODOs: Return Evidence
# TODOs: Neural Language Generation
# TODOs: Better Topic Categorisation, Zero-Shot Text Classification with eternal Knowledge (Concept Net, Hierachy)
# Patirally seen labels (PLL, Partial Label Learning)
# TODOs: Encode Graph

# Does the evidence entail the claim?

# NLI Textual Entailment: Directional relation between text fragments.
# Consider Logical Entailment
# The relation holds whenever the truth of one text fragment follows from another text.
# Text (Premise) -> Hypothesis :: t -> h; t entails h.
# If entails, supports argument
# If not entials, then contradicts the argument

In [417]:
# for i in results_:
#     statements = set()
#     # res = [record for record in nodes.data()]
#
#     for i in range(0, len(results_)):
#         type, relation, concept = results_[i]["e"]
#         # concepts.add(concept)
#
#         concept = list(concept.values())[0].strip("_")
#
#         concept = concept.replace("_", " ")
#
#         if relation == "effect":
#             statements.add(
#                 f"{query} can lead to {concept}"
#             )
#
#         else:
#             statements.add(
#                 f"{concept} can lead to {query}"
#             )

#cause_effect
# list(statements)

['peace can lead to democracy',
 'democracy can lead to tyranny',
 'democracy can lead to peace',
 'democracy can lead to political instability',
 'democracy can lead to chaos',
 'democracy can lead to socialism',
 'economic development can lead to democracy',
 'democracy can lead to prosperity',
 'democracy can lead to instability',
 'capitalism can lead to democracy',
 'economic growth can lead to democracy']